In [ ]:
import numpy as np 
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
import sklearn.tree as tree

my_data = pd.read_csv("sales.csv", delimiter=",")
my_data[0:5]

In [2]:
my_data.shape

(640840, 10)

In [ ]:
my_data.head()


In [ ]:
my_data.isnull().sum() # check for missing values


In [ ]:
my_data.drop(columns=["Unnamed: 0"], inplace=True) #dropping the unnamed column. 


In [18]:
#Converting date.

my_data["date"] = pd.to_datetime(my_data["date"])
my_data["date_int"] = my_data["date"].astype("int64")


In [32]:
my_data["date"] = pd.to_datetime(my_data["date"])
my_data["date"] = pd.to_datetime(my_data["date"])

In [25]:
my_data.drop(columns=["date_int"], inplace=True) 

In [ ]:
# Converting state holiday into integer. 

my_data["state_holiday"] = my_data["state_holiday"].astype("category").cat.codes


In [ ]:
print(my_data.head())

In [ ]:
my_data.dtypes

In [ ]:
print(my_data.describe())

In [ ]:
# Compute correlation matrix
correlation_matrix = my_data.corr()

# Display it
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Matrix")
plt.show()

In [ ]:
sns.boxplot(x="promotion", y="sales", data=my_data)
plt.title("Impact of Promotions on Sales")


In [ ]:
X = my_data.drop(columns=["sales"])  # Drop target variable
y = my_data["sales"]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


